In [1]:
import cv2
import numpy as np
import pytesseract
import re
from bs4 import BeautifulSoup as soup
from Levenshtein import hamming, ratio, distance
from spellchecker import SpellChecker
from nltk import word_tokenize
import os, glob

In [16]:
def preprocess(img):
    """Crop and grayscale."""
    crop = img[100:4300, 100:2800]
    grayscaled = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    return grayscaled

def boundary_box(img):
    """Get boundary boxes of all ROIs in img."""
    blur = cv2.GaussianBlur(img, (15, 15), -1)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernal_e = cv2.getStructuringElement(cv2.MORPH_RECT, (6, 2))
    kernal_d = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 6))
    erode = cv2.erode(thresh, kernal_e, iterations=1)
    dilate = cv2.dilate(erode, kernal_d, iterations=5)
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0] + cv2.boundingRect(x)[1] * img.shape[1])
    
    rectangles = (cv2.minAreaRect(x) for x in cnts)
    boxes = []
    for rectangle in rectangles:
        box = np.int0(cv2.boxPoints(rectangle))
        by_y = sorted(box, key=lambda x: x[1])
        top, bottom = by_y[:2], by_y[2:]
        box = [*sorted(top, key=lambda x: x[0]),*sorted(bottom, key=lambda x: x[0], reverse=True),]
        box = np.array(box)
        boxes.append(box)
    # Sort by x-position
    boxes = sorted(boxes, key=lambda x: (cv2.moments(x)["m10"]/cv2.moments(x)["m00"]))
    # Sort by y-position
    boxes = sorted(boxes, key=lambda x: (cv2.moments(x)["m01"]/cv2.moments(x)["m00"]))
    return boxes

def extract_regions(boxes, ocr_img, lang_custom):
    """Extract desired regions"""
    ocr_results = []
    regions = []
    for n, box in enumerate(boxes):
        box = np.int0(box)
        width, height = get_box_parameters(box)
        if width > 500 and height > 170:
            # Starting region
            src_pts = box.astype("float32")
            # Destination region.
            dst_pts = np.array([[0, 0],[width - 1, 0],[width - 1, height - 1],[0, height - 1],],dtype="float32",)
            transform_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
            warped = cv2.warpPerspective(ocr_img, transform_matrix, (width, height))            
            # https://docs.opencv.org/4.5.3/dc/da3/tutorial_copyMakeBorder.html
            white = [255]
            padded = cv2.copyMakeBorder(warped, 3, 3, 3, 3, cv2.BORDER_CONSTANT, value=white)
            # Save ROI images for diagnostics.
#             cv2.imwrite(f"Test/OCR Test/hOCRfile{i}_{n}.png", padded)
            regions.append(padded)
    return regions

def get_box_parameters(box):
    """Ensures bounding boxes are correctly oriented."""
    rect = cv2.minAreaRect(box)
    height = int(rect[1][1])
    width = int(rect[1][0])
    if abs(rect[2] - 90) < 45: 
        width, height = height, width
    return width, height

def ocr_image(grayscaled):
    """Prepare the input image for the OCR engine"""
    thresh, ocr_img = cv2.threshold(grayscaled, 240, 255, cv2.THRESH_TRUNC)
    return ocr_img

def ocr_region(region, work_name, i):
    """Perform OCR and save transcript as .html file in hOCR format."""
    ocr_result = pytesseract.image_to_pdf_or_hocr(region, lang = lang_custom, extension="hocr")
    with open(f"temp/{work_name}_hocr_{i}.html", "w+b") as f:
        f.write(ocr_result)
        print(f"File {work_name}_{i}.html saved.")
        
def thin_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,1),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

def thick_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,1),np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

def normalise_latin(string):
    """Normalise Latin language character set."""
    norm_string = re.sub("ã|à|ā|â", "a", string)
    norm_string = re.sub("è|ē|ę|ê", "e", norm_string)
    norm_string = re.sub("ì|ī|ĩ|î", "i", norm_string)
    norm_string = re.sub("ĳ", "ii", norm_string)
    norm_string = re.sub("õ|ò|ō|ô", "o", norm_string)
    norm_string = re.sub("ũ|ù|ū|û", "u", norm_string)
    norm_string = re.sub("đ", "d", norm_string)
    norm_string = re.sub("ǣ|ǽ|æ", "ae", norm_string)
    norm_string = re.sub("Æ", "AE", norm_string)
    norm_string = re.sub("œ", "oe", norm_string)
    norm_string = re.sub("Œ", "OE", norm_string)
    norm_string = re.sub("ſ", "s", norm_string)
    norm_string = re.sub("&", "et", norm_string)
    return norm_string

def normalise_greek(string):
    """Remove accidental variant Unicode forms."""
    norm_string = re.sub("\u03d0", "β", string)
    norm_string = re.sub("\u03d1", "θ", norm_string)
    norm_string = re.sub("\u03d2", "Υ", norm_string)
    norm_string = re.sub("\u03d5", "φ", norm_string)
    norm_string = re.sub("\u03d6", "π", norm_string)
    return norm_string

def remove_linebreaks(string):
    """Remove linebreaks and hyphenation from transcript."""
    norm_string = re.sub("\n", "", string)
    norm_string = re.sub("\r\n", "", norm_string)
    norm_string = re.sub("- ", "", norm_string)
    norm_string = re.sub("\|", "", norm_string)
    norm_string = re.sub(" (\.|,|;|:|!|\?)", "\g<1>", norm_string)
    return norm_string

def hocr_to_text(file_number, work_name):
    """Load and parse the hOCR file."""
    transcript_list = []
    for i in range(file_number):
        with open(f"temp/{work_name}_hocr_{i}.html", encoding = "utf-8") as f:
            hocr = soup(f, "html.parser")
        ocr_pars = hocr.find_all(class_="ocr_carea")
        for j, section in enumerate(ocr_pars):
            section = ocr_pars[j].get_text(" ",strip=True)
            if len(section) > 5:
                transcript_list.append(section)
    return transcript_list

def sequence_check(transcript_list):
    """Prints beginning and end of each section. Use to check ROIs are in correct order."""
    for i, section in enumerate(transcript_list):
        print(f"Section {i} length: {len(section)}\nSection {i} incipit: {section[0:60]}\nSection {i} explicit: {section[-60:]}\n=========")

def spell_check(text):
    """Tokenise transcript and run Pyspellcheck."""
    spell_checked = []
    tokens = word_tokenize(text)
    for token in tokens:
        if re.search("[A-Za-z]", token):
            token_corrected = spell_lat.correction(token)
            spell_checked.append(token_corrected)
            print(token, "\t → \t", token_corrected)
        elif re.search("[\u0370-\u1fff]", token):
            token_corrected = spell_grc.correction(token)
            spell_checked.append(token_corrected)
            print(token, "\t → \t", token_corrected)
    spell_checked = " ".join(spell_checked)
    return spell_checked

In [13]:
"""Set variables."""
pdf_number = "150"
work_name = "116-xenophon"
lang_custom = r"lat+grc_all"
folder_path = "input/"

"""Fetch input file names."""
pages = []
for filename in glob.glob(os.path.join(folder_path, f"{pdf_number}*?.png")):
    pages.append(filename)
print(pages)

['input\\150 Xenophon-1.png', 'input\\150 Xenophon-2.png', 'input\\150 Xenophon-3.png', 'input\\150 Xenophon-4.png', 'input\\150 Xenophon-5.png', 'input\\150 Xenophon-6.png']


In [17]:
"""Run OCR program."""
regions = []
for i, page in enumerate(pages):
    file_string = pages[i]
    img = cv2.imread(file_string)
    grayscaled = preprocess(img)
    boxes = boundary_box(grayscaled)
    ocr_img = ocr_image(grayscaled)
    region = extract_regions(boxes, ocr_img, lang_custom)
    regions.append(region)
regions = [a for b in regions for a in b]
file_number = len(regions)
for i, region in enumerate(regions):
    ocr_region(region, work_name, i)
print("OCR finished.")

File 116-xenophon_0.html saved.
File 116-xenophon_1.html saved.
File 116-xenophon_2.html saved.
File 116-xenophon_3.html saved.
File 116-xenophon_4.html saved.
File 116-xenophon_5.html saved.
File 116-xenophon_6.html saved.
File 116-xenophon_7.html saved.
File 116-xenophon_8.html saved.
File 116-xenophon_9.html saved.
File 116-xenophon_10.html saved.
File 116-xenophon_11.html saved.
File 116-xenophon_12.html saved.
File 116-xenophon_13.html saved.
File 116-xenophon_14.html saved.
File 116-xenophon_15.html saved.
File 116-xenophon_16.html saved.
File 116-xenophon_17.html saved.
OCR finished.


In [19]:
transcript_list = hocr_to_text(file_number, work_name)
sequence_check(transcript_list)

Section 0 length: 3617
Section 0 incipit: IMONIDES PoETA die quodam acceſſit ad Hieronem Tyrannum, Juu
Section 0 explicit: me ſpoctatu dignas, ſiſdem licet accedere. Tyranni porro non
Section 1 length: 3860
Section 1 incipit: multum indulgent ſpectaculis, raroque interſunt, eo quod ipſ
Section 1 explicit: odoribus, inquit Simonides, iſtis pretioſis quibus un- gimi-
Section 2 length: 9
Section 2 incipit: 645 Η 1Ὲ8
Section 2 explicit: 645 Η 1Ὲ8
Section 3 length: 4285
Section 3 incipit: gimini, arbitror, eos plus boni capere, qui ſunt familiares,
Section 3 explicit: irtute præſtantes, tum arma inſigniter pulera, mundum eximi-
Section 4 length: 8
Section 4 incipit: r Oæ 646
Section 4 explicit: r Oæ 646
Section 5 length: 4271
Section 5 incipit: ſum uxoribus, ædes magnificentiſſimas, eaſque rebus pre- tio
Section 5 explicit: icitias, quarum m ſunt Tyranni. Primum enim quoniam magnum \
Section 6 length: 68
Section 6 incipit: onum hominibus eſt amicitia, hoc conſideremus; Et- Tom. IV. 
Section

In [20]:
"""Commands for ammending transcript ordering and removing noise."""
print("Original length: ",len(transcript_list))
# Change order
transcript_list[6], transcript_list[7] = transcript_list[7], transcript_list[6]
transcript_list[13], transcript_list[16] = transcript_list[16], transcript_list[13]
transcript_list[18], transcript_list[17] = transcript_list[17], transcript_list[18]
transcript_list[19], transcript_list[23] = transcript_list[23], transcript_list[19]
transcript_list[26], transcript_list[29] = transcript_list[29], transcript_list[26]
transcript_list[36], transcript_list[38] = transcript_list[38], transcript_list[36]
transcript_list[47], transcript_list[46] = transcript_list[46], transcript_list[47]
transcript_list[81], transcript_list[80] = transcript_list[80], transcript_list[81]
transcript_list[87], transcript_list[86] = transcript_list[86], transcript_list[87]
print("New length: ",len(transcript_list))
# Remove noise
del transcript_list[34]
del transcript_list[32]
del transcript_list[30]
del transcript_list[28]
del transcript_list[27]
del transcript_list[25]
del transcript_list[20]
del transcript_list[18]
del transcript_list[16]
del transcript_list[14]
del transcript_list[13]
del transcript_list[9]
del transcript_list[6]
del transcript_list[4]
del transcript_list[2]
print("New length: ",len(transcript_list))

Original length:  54
New length:  54
New length:  39


In [21]:
sequence_check(transcript_list)

Section 0 length: 3617
Section 0 incipit: IMONIDES PoETA die quodam acceſſit ad Hieronem Tyrannum, Juu
Section 0 explicit: me ſpoctatu dignas, ſiſdem licet accedere. Tyranni porro non
Section 1 length: 3860
Section 1 incipit: multum indulgent ſpectaculis, raroque interſunt, eo quod ipſ
Section 1 explicit: odoribus, inquit Simonides, iſtis pretioſis quibus un- gimi-
Section 2 length: 4285
Section 2 incipit: gimini, arbitror, eos plus boni capere, qui ſunt familiares,
Section 2 explicit: irtute præſtantes, tum arma inſigniter pulera, mundum eximi-
Section 3 length: 4271
Section 3 incipit: ſum uxoribus, ædes magnificentiſſimas, eaſque rebus pre- tio
Section 3 explicit: icitias, quarum m ſunt Tyranni. Primum enim quoniam magnum \
Section 4 length: 6
Section 4 incipit: 645 Hu
Section 4 explicit: 645 Hu
Section 5 length: 4285
Section 5 incipit: gimini, arbitror, eos plus boni capere, qui ſunt familiares,
Section 5 explicit: irtute præſtantes, tum arma inſigniter pulera, mundum eximi-
Section

In [22]:
transcript = " ".join(transcript_list)
transcript = normalise_latin(transcript)
transcript = normalise_greek(transcript)
transcript = remove_linebreaks(transcript)
with open(f"temp/{work_name}_transcript.txt", "w", encoding = "utf-8") as f:
    f.write(transcript)
    print(f"File {work_name}_transcript.txt saved.")

File 116-xenophon_transcript.txt saved.


Postprocessing

In [23]:
"""Load Latin and Greek frequency lists."""
spell_lat = SpellChecker(local_dictionary="latin_freq.json")
spell_grc = SpellChecker(local_dictionary="greek_freq.json")

In [24]:
with open(f"temp/{work_name}_transcript.txt", "r", encoding = "utf-8") as f:
    text = f.read()
spell_checked = spell_check(text)
spell_checked_2 = re.sub(" (\.|,|;|:|\?|\!|·|;)", "\g<1>", spell_checked)
with open(f"output/{work_name}_spellcheck.txt", "w", encoding = "utf-8") as f:
    f.write(spell_checked_2)
    print(f"File {work_name}_spellcheck.txt saved.")

Xenophontis 	 → 	 Xenophontis
Rhetoris 	 → 	 Rhetoris
Hieron 	 → 	 Hieron
sive 	 → 	 sive
Tyrannus 	 → 	 Tyrannus
Desiderio 	 → 	 Desiderio
Erasmo 	 → 	 Erasmo
Roterodamo 	 → 	 roterodamus
interprete 	 → 	 interprete
Simonides 	 → 	 Simonides
poeta 	 → 	 poeta
die 	 → 	 die
quodam 	 → 	 quodam
accessit 	 → 	 accessit
ad 	 → 	 ad
Hieronem 	 → 	 Hieronem
Tyrannum 	 → 	 Tyrannum
Juumque 	 → 	 quumque
otium 	 → 	 otium
forte 	 → 	 forte
esset 	 → 	 esset
utrisque 	 → 	 utrisque
Simonides 	 → 	 Simonides
hunc 	 → 	 hunc
in 	 → 	 in
modum 	 → 	 modum
locutus 	 → 	 locutus
cst 	 → 	 est
Vellesne 	 → 	 vellent
mihi 	 → 	 mihi
explicare 	 → 	 explicare
Hieron 	 → 	 Hieron
ῃ 	 → 	 ὁ
v 	 → 	 v
ea 	 → 	 ea
quae 	 → 	 quae
probabile 	 → 	 probabile
est 	 → 	 est
te 	 → 	 te
melius 	 → 	 melius
nosse 	 → 	 nosse
ah 	 → 	 ah
quam 	 → 	 quam
ego 	 → 	 ego
novi 	 → 	 novi
Et 	 → 	 Et
quaenam 	 → 	 quaenam
inquit 	 → 	 inquit
Hieron 	 → 	 Hieron
sunt 	 → 	 sunt
ista 	 → 	 ista
quae 	 → 	 quae
mihi 	 → 	

spoctatu 	 → 	 spectat
dignas 	 → 	 dignas
sisdem 	 → 	 isdem
licet 	 → 	 licet
accedere 	 → 	 accedere
Tyranni 	 → 	 Tyranni
porro 	 → 	 porro
non 	 → 	 non
multum 	 → 	 multum
indulgent 	 → 	 indulgent
spectaculis 	 → 	 spectaculis
raroque 	 → 	 raroque
intersunt 	 → 	 intersunt
eo 	 → 	 eo
quod 	 → 	 quod
ipsis 	 → 	 ipsis
eo 	 → 	 eo
proficisci 	 → 	 proficisci
tutum 	 → 	 tutum
non 	 → 	 non
est 	 → 	 est
ubi 	 → 	 ubi
non 	 → 	 non
sint 	 → 	 sint
futuri 	 → 	 futuri
superiores 	 → 	 superiores
iis 	 → 	 iis
quiadsunt 	 → 	 quiadsunt
Nec 	 → 	 Nec
quae 	 → 	 quae
domi 	 → 	 domi
possident 	 → 	 possident
in 	 → 	 in
tuto 	 → 	 tuto
sunt 	 → 	 sunt
ut 	 → 	 ut
his 	 → 	 his
apud 	 → 	 apud
alios 	 → 	 alios
depositis 	 → 	 depositis
liceat 	 → 	 liceat
abesse 	 → 	 abesse
peregre 	 → 	 peregre
Metuendum 	 → 	 Metuendum
enim 	 → 	 enim
est 	 → 	 est
ne 	 → 	 ne
u 	 → 	 u
et 	 → 	 et
Principatu 	 → 	 Principatu
excutiantur 	 → 	 excutiuntur
et 	 → 	 et
facultatem 	 → 	 facultatem
am

illubentius 	 → 	 lubentius
ut 	 → 	 ut
vulgo 	 → 	 vulgo
quidem 	 → 	 quidem
videri 	 → 	 videri
possit 	 → 	 possit
Tum 	 → 	 Tum
Hieron 	 → 	 Hieron
Quid 	 → 	 Quid
inquit 	 → 	 inquit
num 	 → 	 num
ista 	 → 	 ista
multo 	 → 	 multo
studio 	 → 	 studio
arteque 	 → 	 arteque
confecta 	 → 	 confecta
considerasti 	 → 	 consideranti
quae 	 → 	 quae
Regibus 	 → 	 Regibus
adponuntur 	 → 	 adponuntur
acria 	 → 	 acria
amara 	 → 	 amara
austeraque 	 → 	 alteraque
et 	 → 	 et
his 	 → 	 his
germana 	 → 	 germana
Prorsus 	 → 	 Prorsus
inquit 	 → 	 inquit
Simonides 	 → 	 Simonides
consideravi 	 → 	 consideravi
Planeque 	 → 	 Planeque
mihi 	 → 	 mihi
quidem 	 → 	 quidem
ista 	 → 	 ista
videntur 	 → 	 videntur
esse 	 → 	 esse
praeter 	 → 	 praeter
naturam 	 → 	 naturam
hominibus 	 → 	 hominibus
adscita 	 → 	 adscita
Quid 	 → 	 Quid
igitur 	 → 	 igitur
inquit 	 → 	 inquit
Hieron 	 → 	 Hieron
aliud 	 → 	 aliud
existimas 	 → 	 existimas
hoc 	 → 	 hoc
genus 	 → 	 genus
epulas 	 → 	 epulas
esse 	 → 	 

Dailocham 	 → 	 Dailocham
ob 	 → 	 ob
quaecumque 	 → 	 quaecumque
sunt 	 → 	 sunt
illa 	 → 	 illa
quae 	 → 	 quae
fortassis 	 → 	 fortassis
natura 	 → 	 natura
hominis 	 → 	 hominis
cogit 	 → 	 cogit
a 	 → 	 ad
formosis 	 → 	 formosis
petere 	 → 	 petere
Caeterum 	 → 	 Caeterum
his 	 → 	 his
potiri 	 → 	 potiri
quae 	 → 	 quae
amo 	 → 	 amo
ut 	 → 	 ut
cum 	 → 	 cum
mutua 	 → 	 mutua
benevolentia 	 → 	 benevolentia
et 	 → 	 et
a 	 → 	 ad
volente 	 → 	 volente
contingat 	 → 	 contingat
vehementer 	 → 	 vehementer
opto 	 → 	 opto
Ut 	 → 	 Ut
autem 	 → 	 autem
vi 	 → 	 vi
ab 	 → 	 ab
illa 	 → 	 illa
sumam 	 → 	 sumam
minus 	 → 	 minus
mihi 	 → 	 mihi
videor 	 → 	 videor
concupiscere 	 → 	 concupiscere
quam 	 → 	 quam
concuisco 	 → 	 concupisco
mihi 	 → 	 mihi
ipsi 	 → 	 ipsi
mali 	 → 	 mali
quidpiam 	 → 	 quidpiam
facere 	 → 	 facere
Nam 	 → 	 Nam
ab 	 → 	 ab
invitis 	 → 	 invitis
hostialiquid 	 → 	 hostialiquid
sumere 	 → 	 sumere
rem 	 → 	 rem
equidem 	 → 	 equidem
arbitror 	 → 	 arbitr

formidantibus 	 → 	 formidantibus
ne 	 → 	 ne
quis 	 → 	 quis
ipsos 	 → 	 ipsos
interficiat 	 → 	 interficiat
Tyranni 	 → 	 Tyranni
autem 	 → 	 autem
omnes 	 → 	 omnes
quoquo 	 → 	 quoquo
se 	 → 	 se
conferant 	 → 	 conferant
quasi 	 → 	 quasi
per 	 → 	 per
hostilia 	 → 	 hostilia
castra 	 → 	 castra
proficiscuntur 	 → 	 proficiscuntur
Eoque 	 → 	 Eoque
necesse 	 → 	 necesse
purant 	 → 	 putant
ut 	 → 	 ut
et 	 → 	 et
ipsi 	 → 	 ipsi
semper 	 → 	 semper
sint 	 → 	 sint
armati 	 → 	 armati
et 	 → 	 et
alios 	 → 	 alios
armigeros 	 → 	 armigeros
jugiter 	 → 	 jugiter
secum 	 → 	 secum
circumferant 	 → 	 circumferunt
Praeterea 	 → 	 Praeterea
privati 	 → 	 privati
etiam 	 → 	 etiam
si 	 → 	 si
quando 	 → 	 quando
proficiscantur 	 → 	 proficiscantur
in 	 → 	 in
expeditionem 	 → 	 expeditionem
tamen 	 → 	 tamen
simul 	 → 	 simul
atque 	 → 	 atque
domum 	 → 	 domum
se 	 → 	 se
receperint 	 → 	 receperint
existimant 	 → 	 existimant
sibi 	 → 	 sibi
res 	 → 	 res
esse 	 → 	 esse
tutas 	 → 	 tu

...

expavescentem 	 → 	 expavescentem
atque 	 → 	 atque
attonitum 	 → 	 attonitum
et 	 → 	 et
unde 	 → 	 unde
periculum 	 → 	 periculum
sit 	 → 	 sit
ne 	 → 	 ne
quod 	 → 	 quod
malum 	 → 	 malum
det 	 → 	 det
immedicabile 	 → 	 immedicabile
ut 	 → 	 ut
gravate 	 → 	 gravate
occidit 	 → 	 occidit
illum 	 → 	 illum
ob 	 → 	 ob
virtutem 	 → 	 virtutem
ita 	 → 	 ita
aegre 	 → 	 aegre
potest 	 → 	 potest
illo 	 → 	 illo
uti 	 → 	 uti
observans 	 → 	 observans
ac 	 → 	 ac
sollicitus 	 → 	 sollicitus
ne 	 → 	 ne
quod 	 → 	 quod
in 	 → 	 in
periculis 	 → 	 periculis
adferat 	 → 	 adferat
exitiale 	 → 	 exitiale
malum 	 → 	 malum
Itidem 	 → 	 Itidem
et 	 → 	 et
reliquae 	 → 	 reliquae
possessiones 	 → 	 possessiones
quae 	 → 	 quae
sic 	 → 	 sic
periculosae 	 → 	 periculosae
sunt 	 → 	 sunt
ut 	 → 	 ut
sint 	 → 	 sint
utiles 	 → 	 utiles
molestiam 	 → 	 molestiam
adferunt 	 → 	 adferunt
possidenti 	 → 	 possidenti
rursum 	 → 	 rursum
molestiam 	 → 	 molestiam
adferunt 	 → 	 adferunt
abjicienti 	 →

rependendam 	 → 	 defendendam
iis 	 → 	 iis
quos 	 → 	 quos
spoliavit 	 → 	 spoliavit
aut 	 → 	 aut
quomodo 	 → 	 quomodo
vincula 	 → 	 vincula
repenset 	 → 	 recenset
his 	 → 	 his
quos 	 → 	 quos
detrusit 	 → 	 detrusit
in 	 → 	 in
vincula 	 → 	 vincula
aut 	 → 	 aut
quomodo 	 → 	 quomodo
restituet 	 → 	 restituet
tor 	 → 	 tot
animas 	 → 	 animas
exstinctas 	 → 	 exstincta
his 	 → 	 his
quos 	 → 	 quos
occidit 	 → 	 occidit
Sed 	 → 	 Sed
si 	 → 	 si
cuiquam 	 → 	 cuiquam
alteri 	 → 	 alteri
o 	 → 	 o
Simonides 	 → 	 Simonides
expedit 	 → 	 expedit
laqueo 	 → 	 laqueo
finire 	 → 	 finire
vitam 	 → 	 vitam
scito 	 → 	 scito
inquit 	 → 	 inquit
me 	 → 	 me
compertum 	 → 	 compertum
habere 	 → 	 habere
ut 	 → 	 ut
id 	 → 	 id
faciat 	 → 	 faciat
nulli 	 → 	 nulli
magis 	 → 	 magis
expedire 	 → 	 expedire
quam 	 → 	 quam
Tyranno 	 → 	 Tyranno
quandoquidem 	 → 	 quandoquidem
huic 	 → 	 huic
uni 	 → 	 uni
masa 	 → 	 mala
nec 	 → 	 nec
retinere 	 → 	 retinere
nec 	 → 	 nec
deponere 	 → 	 de

conductitio 	 → 	 conductio
milite 	 → 	 milite
opus 	 → 	 opus
Tom 	 → 	 Tom
IV 	 → 	 IV
Ss3 	 → 	 ss
es 	 → 	 es
Aaliqui 	 → 	 aliqui
facinorosi 	 → 	 facinoris
in 	 → 	 in
civitatibus 	 → 	 civitatibus
Itaque 	 → 	 Itaque
si 	 → 	 si
fuerit 	 → 	 fuerit
imperatum 	 → 	 imperatum
ut 	 → 	 ut
satellites 	 → 	 satellites
et 	 → 	 et
illos 	 → 	 illos
custodiant 	 → 	 custodiant
et 	 → 	 et
jam 	 → 	 jam
in 	 → 	 in
hoc 	 → 	 hoc
cives 	 → 	 cives
intelligerent 	 → 	 intelligerent
illos 	 → 	 illos
sibi 	 → 	 sibi
esse 	 → 	 esse
utiles 	 → 	 utiles
Praeterea 	 → 	 Praeterea
ut 	 → 	 ut
tueantur 	 → 	 tueantur
et 	 → 	 et
hos 	 → 	 hos
qui 	 → 	 qui
in 	 → 	 in
agris 	 → 	 agris
operantur 	 → 	 operantur
et 	 → 	 et
jumenta 	 → 	 jumenta
hi 	 → 	 hi
nimirum 	 → 	 nimirum
ut 	 → 	 ut
est 	 → 	 est
probabile 	 → 	 probabile
tum 	 → 	 tum
securitatem 	 → 	 securitatem
tum 	 → 	 tum
fiduciam 	 → 	 fiduciam
praesiare 	 → 	 praestare
possent 	 → 	 possent
simul 	 → 	 simul
et 	 → 	 et
tuis 	 

ficentissimoque 	 → 	 ficentissimoque
certaImine 	 → 	 certamine
victorem 	 → 	 victorem
sore 	 → 	 ore
Atque 	 → 	 Atque
in 	 → 	 in
primis 	 → 	 primis
quidem 	 → 	 quidem
illud 	 → 	 illud
effeceris 	 → 	 effeceris
ut 	 → 	 ut
ameris 	 → 	 ameris
ab 	 → 	 ab
his 	 → 	 his
quibus 	 → 	 quibus
imperas 	 → 	 imperas
cujus 	 → 	 cujus
sane 	 → 	 sane
rei 	 → 	 rei
tu 	 → 	 tu
perscupiduses 	 → 	 perscupiduses
tum 	 → 	 tum
vero 	 → 	 vero
victoriae 	 → 	 victoriae
tuae 	 → 	 tuae
nonunum 	 → 	 nondum
habiturus 	 → 	 habiturus
es 	 → 	 es
praeconem 	 → 	 praeconem
sed 	 → 	 sed
omnes 	 → 	 omnes
homines 	 → 	 homines
laudibus 	 → 	 laudibus
vehent 	 → 	 vehunt
virtutem 	 → 	 virtutem
tuam 	 → 	 tuam
Insuper 	 → 	 Insuper
omnibus 	 → 	 omnibus
suspiciendus 	 → 	 suspiciens
non 	 → 	 non
tantum 	 → 	 tantum
a 	 → 	 ad
privatis 	 → 	 privatis
verum 	 → 	 verum
etiam 	 → 	 etiam
a 	 → 	 ad
multis 	 → 	 multis
civitatibusdiligeris 	 → 	 civitatibusdiligeris
nec 	 → 	 nec
privatim 	 → 	 privat

vincendique 	 → 	 vendendique
studium 	 → 	 studium
Ac 	 → 	 Ac
per 	 → 	 per
Jovem 	 → 	 Jovem
citius 	 → 	 citius
impelluntur 	 → 	 impelluntur
quocumque 	 → 	 quocumque
opus 	 → 	 opus
est 	 → 	 est
qui 	 → 	 qui
honoris 	 → 	 honoris
suntavidi 	 → 	 suntavidi
citiusque 	 → 	 citiusque
conferrent 	 → 	 conferrent
pecuniam 	 → 	 pecuniam
si 	 → 	 si
quando 	 → 	 quando
pecuniam 	 → 	 pecuniam
exigat 	 → 	 exigat
occasio 	 → 	 occasio
Et 	 → 	 Et
quod 	 → 	 quod
est 	 → 	 est
omnium 	 → 	 omnium
utilissimum 	 → 	 utilissimum
id 	 → 	 id
minime 	 → 	 minime
tamen 	 → 	 tamen
usitatum 	 → 	 usitatum
est 	 → 	 est
ut 	 → 	 ut
per 	 → 	 per
certamen 	 → 	 certamen
exerceatur 	 → 	 exerceatur
agricultura 	 → 	 agricultura
Haec 	 → 	 Haec
nimirum 	 → 	 nimirum
est 	 → 	 est
cui 	 → 	 cui
multum 	 → 	 multum
accesserit 	 → 	 accesserit
si 	 → 	 si
quis 	 → 	 quis
per 	 → 	 per
agros 	 → 	 agros
ac 	 → 	 ac
vicos 	 → 	 vicos
praemia 	 → 	 praemia
constituat 	 → 	 constituat
his 	 → 	 his
qui 

Ground truth 

In [ ]:
with open("Test/OCR Test/GT/Querela Pacis GT.txt", "r", encoding = "utf-8") as text_file:
    gt = text_file.read()

In [ ]:
print("GT length:", len(gt), "OCR transcript length:", len(transcript))
print("Levenshtein:", distance(gt, transcript))
print("Ratio:", ratio(gt, transcript))

HTML Encoding

In [26]:
WORK = "116-xenophon"
TITLES = {"116": "Xenophon"}

SRC = f"output/{WORK}_spellcheck.txt"
DEST = f"output/{WORK}.html"

TITLE = TITLES[WORK[:3]]

HEADER = f"""\
<!DOCTYPE html>
<html lang="lat">
<head>
<title>{TITLE}</title>
<meta charset="utf-8">
<link href="https://fonts.googleapis.com/css?family=Noto+Serif:400,700&amp;subset=greek,greek-ext" rel="stylesheet">
<link href="style.css" rel="stylesheet">
</head>
<body>
  <div class="container">
  <nav>&#x2191; <a href="./">Corpus Erasmicum</a></nav>
  <h1 lang="en">{TITLE}</h1>
"""

FOOTER = """\
  </div>
</body>
</html>
"""

with open(SRC, encoding = "utf-8") as f:
    with open(DEST, "w", encoding = "utf-8") as g:
        prev_section = None
        prev_chapter = None
        print(HEADER, file=g)
        for line in f:
            parts = line.strip().split(maxsplit=1)
            ref = parts[0].split(".")
            if len(ref) == 2:
                section = None
                chapter, verse = ref
            if prev_section != section:
                if prev_section is not None:
                    print("   </div>""", file=g)
                    print("   </div>""", file=g)
                print("""   <div class="section">""", file=g)
                prev_section = section
                prev_chapter = None
            if prev_chapter != chapter:
                if prev_chapter is not None: 
                    if prev_chapter == "0":
                        if section is None:
                            print("""    </div>""", file=g)
                    else:
                        print("""    </div>""", file=g)
                if chapter == "0":
                    if section is None:
                        print("""    <div class="preamble">""", file=g)
                else:
                    if chapter == "SB":
                        print("""    <div class="subscription">""", file=g)
                    elif chapter == "EP":
                        print("""    <div class="epilogue">""", file=g)
                    else:
                        print("""    <div class="chapter">""", file=g)

                prev_chapter = chapter
            if chapter == "0" and verse == "0":
                print(f"""    <h2 class="section_title">{parts[1]}</h2>""", file=g)
            elif chapter != "0" and verse == "0":
                print(f"""<h3 class="sub_title">{parts[1]}</h3>""", file=g)
            elif chapter != "0" and verse != "0":
                print(parts[1], file=g)
        print("""    </div>""", file=g)
        if section is not None:
            print("""    </div>""", file=g)
        print(FOOTER, file=g)